1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-lieve of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import scipy as sc

In [ ]:
n_Ti=1000
n_Pb=0
tau=3.052*60 #seconds

def p(t):
    return 1-2**(-t/tau)

In [ ]:
#time evolution
time=1000
Ti=[]
Pb=[]

p_t=p(1)
print(p_t)

for t in range(1,time):
    Ti.append(n_Ti)
    Pb.append(n_Pb)
    for j in range(n_Ti):
        if np.random.random()<=p_t:
            n_Ti=n_Ti-1
            n_Pb=n_Pb+1
    
t=np.arange(0, len(Ti), 1)

In [ ]:
plt.figure()
plt.plot(t, Ti, Pb)
plt.legend(['Ti', 'Pb'])
plt.xlabel('Time[s]')
plt.ylabel('Population')
plt.grid()
plt.show()

In [ ]:
time=np.linspace(0, 2000)

def p2(t):
    return 2**(-t/tau)*np.log(2)/tau

def icp2(t):
    return -tau*np.log2(1-t)

plt.plot(time, p2(time))

t_unif=np.random.rand(1000)
v=icp2(t_unif)
plt.hist(v, histtype='step', bins=100, density=True, linewidth=2)
plt.ylabel('counts')
plt.xlabel('time')
plt.show()


2\. **Rutherford Scattering**

The scattering angle $\theta$ of $\alpha$ particles hitting a positively charged nucleus of a Gold atom ($Z=79$) follows the rule:

$$
\tan{\frac{1}{2} \theta} = \frac{Z e^2} {2\pi \epsilon_0 E b}
$$

where $E=7.7$ MeV and $b$ beam is the impact parameter. The beam is represented by a 2D gaussian distribution with $\sigma=a_0/100$ for both coordinates ($a_0$ being the Bohr radius). Assume 1 million $\alpha$ particles are shot on the gold atom.

Computing the fraction of particles that "bounce back",i.e. those particle whose scattering angle is greater than $\pi/2$ (which set a condition on the impact parameter $b$)

In [ ]:
E = 7.7 * 10**6
a0 = 137 * 2 *10**(-6)
Z = 79
e = 1
ep0 = 1./(4*np.pi)

sigma = a0/100.
N=1000000


K=Z*e*e/(2*np.pi*ep0*E)

In [ ]:
beam = np.random.multivariate_normal([0, 0], np.diag([sigma, sigma]), size=N)

b = np.sqrt(beam[:,0]**2 + beam[:,1]**2)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(beam[:, 0], beam[:, 1], s=1)
plt.grid()
plt.show()

In [ ]:
bounce = (b<=K)
n = bounce.sum()
print(bounce.sum())
print(n/1000000)

3\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function 

$$f(x) =\sin^2{\frac{1}{x(2-x)}}$$

* Compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one

In [ ]:
def f(x):
    return (np.sin(1/(x*(2-x))))**2

In [ ]:
x = np.linspace(10**(-3), 1.999, 1000)
y = f(x)

In [ ]:
#hit/miss
N = 100000
count=0
for i in range(N):
    x=2*np.random.random()
    y=np.random.random()
    if y<f(x): count+=1
Ic=2*count/N
print(Ic)

In [ ]:
#mean value
x=np.random.random(N)
f_i=f(x)

I=2./N*f_i.sum()
print(I)

4\. **Monte Carlo integration in high dimension**

* Start of by computing the area of a circle of unit radius, by integrating the function 

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere



In [9]:
def f(x,y):
    if (x*x+y*y<=1) :
        out=1
    else:
        out=0
    return out

In [15]:
N=1000
x=np.random.uniform(-1,1,N)
y=np.random.uniform(-1,1,N)

count=0
for i in range(N):
    count+=f(x[i], y[i])

#hit/miss
Ic=4*count/N
print(Ic)

3.192


In [32]:
def f(x):
    if np.sum([i**2 for i in x])<=1:
        out=1
    else:
        out=0
    return out

In [34]:
# 10 D sphere

N=1000000
d=10

V=2**10

x=np.random.uniform(-1,1, (N, d))
count=0
for i in range(N):
    count+=f(x[i,:])

Ic=V*count/N
print(Ic)

2.545664


In [35]:
#True values
print(np.pi**5/120)

2.550164039877345


5\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should get a result about 0.84

In [37]:
def f(x):
    return (x**(-0.5))/(np.exp(x)+1)

def w(x):
    return 1./np.sqrt(x)

def intw(x):   
    return 2*np.sqrt(x)

In [42]:
N = 10000000
x = np.random.random(N)
I = (1/N)* (f(x)/w(x)).sum()*(intw(1)-intw(0.000001))
print(I)

0.7590000984002523
